# Tiny few shot classifer

Im folgenden wird ein few shot classifier im [SetFit-Frame](https://huggingface.co/blog/setfit) work erzeugt.

## Vorteile von SetFit

Die Verwendung von SetFit (Sentence Transformer Fine-tuning) hat mehrere Vorteile für die hiesige Anwendung:

1. **Few-Shot-Learning**: SetFit wurde speziell entwickelt, um mit sehr wenigen Trainingsbeispielen pro Klasse zu arbeiten (z. B. 8 Beispiele). Das Modell erreicht dennoch eine hohe Genauigkeit, was es ideal für Anwendungsfälle mit begrenzten annotierten Daten macht.

2. **Effiziente Fine-Tuning-Methode**: Anstatt ein großes Sprachmodell vollständig zu trainieren, verwendet SetFit einen zweistufigen Ansatz. Zuerst wird ein vortrainiertes Sentence Transformer-Modell (z. B. MiniLM) auf die Textdaten angepasst, dann wird ein einfacher Klassifikator darauf angewendet. Dies ist sowohl rechen- als auch speichereffizient.

3. **Keine zusätzliche Tokenisierung**: SetFit arbeitet direkt mit Sentence Embeddings, was bedeutet, dass keine zusätzliche Tokenisierung oder Verarbeitung für die Fine-Tuning-Aufgabe erforderlich ist. Dies vereinfacht den Einsatz und spart Rechenressourcen.

4. **Gute Performance bei kleinerem Modell**: Durch die Verwendung von kompakten Sentence Transformers kann SetFit eine hohe Genauigkeit erreichen, ohne die Ressourcenanforderungen eines größeren Modells zu benötigen.

5. **Schnelle Anpassung**: Aufgrund des effizienten Fine-Tuning-Prozesses kann SetFit in kürzerer Zeit auf neue Datensätze angepasst werden, was es für den produktiven Einsatz in schnell wechselnden Umgebungen geeignet macht.

# Berechnung des few shot Modells

## Lade Bibliotheken

Folgende Bibliotheken werden geladen:

- `from datasets import load_dataset`: Zum Laden von Datensätzen aus der HuggingFace `datasets` Bibliothek, z.B. öffentliche oder lokale Datensätze.
  
- `from sentence_transformers.losses import CosineSimilarityLoss`: Wird verwendet, um einen *Cosine Similarity Loss* für das Training von sentence-transformers-Modellen zu definieren (z.B. bei Ähnlichkeitsvergleichen zwischen Textpaaren).
  
- `from setfit import SetFitModel, SetFitTrainer`:  
  - **SetFitModel**: Ein Modell aus dem SetFit-Framework, das für Zero-Shot- oder Few-Shot-Learning verwendet wird.
  - **SetFitTrainer**: Trainer-Klasse, die verwendet wird, um das SetFit-Modell auf spezifische Aufgaben zu trainieren.
  
- `import pandas as pd`: Für die Data-Wrangling
  
- `from sklearn.preprocessing import LabelEncoder`: Zum Kodieren von Zielvariablen (Labels) in numerische Form. Besonders nützlich, wenn die Labels in string-Format vorliegen und in Zahlen umgewandelt werden müssen, um sie einem Modell zu übergeben.
  
- `import os`: Ermöglicht die Interaktion mit dem Betriebssystem, wie z.B. den Zugriff auf Dateipfade, Erstellen von Ordnern oder Lesen von Umgebungsvariablen.
  
- `from datasets import Dataset, DatasetDict`:  
  - **Dataset**: Klasse, um einen benutzerdefinierten Datensatz für die Verwendung mit der `datasets` Bibliothek zu erstellen.
  - **DatasetDict**: Zum Organisieren und Verwalten mehrerer Datensätze (z.B. `train` und `test`).
  
- `from sklearn.model_selection import train_test_split`: Zum Aufteilen eines Datensatzes in Trainings- und Testdaten. Diese Methode wird oft verwendet, um Modelle zu validieren und zu evaluieren.
  
- `from transformers import TrainingArguments`: Legt die Trainingsparameter für Modelle aus der HuggingFace Transformers Bibliothek fest (z.B. Anzahl der Epochen, Batch-Größe, Optimierungsparameter etc.).


In [11]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import os
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

from transformers import TrainingArguments

##  Daten
### Lade Daten

Um die Daten zu laden wird in einem ersten Schritt das Root-Verzeichnis ausgelesen und ggf. angepasst:

In [12]:
# checke home-verzeichnis
print(os.getcwd())
# setze home-verzeichnis
os.chdir('c:/Users/Hueck/OneDrive/Dokumente/GitHub/future_skill_classification/')

c:\Users\Hueck\OneDrive\Dokumente\GitHub\future_skill_classification


Stimmt das Verzeichnis, kann im folgenden der Datensatz geladen werden. Wir laden die Daten und schauen uns die ersten

In [14]:
# Laden Ihrer Excel-Datei
df = pd.read_excel('data/fs_test_data.xlsx')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   text        637 non-null    object
 1   label       637 non-null    int64 
 2   label_text  637 non-null    object
dtypes: int64(1), object(2)
memory usage: 15.1+ KB


### Erzeuge Test- und Trainingsdaten

Im folgenden wird der Datensatz für das Training und die Evaluierung des Clasifiers vorbereitet. 

1. Der Data Frame wird in zwei Teile geteilt: Trainingsdaten (train_df) und Testdaten (test_df). 20% der Daten werden für den Testdatensatz verwendet, während 80% als Trainingsdatensatz verwendet werden. Mit 
`random_state=42` wird sicher gestellt, dass die Aufteilung reproduzierbar ist. 
`stratify=df['label']` sichert schließlich, dass die Verteilung der Zielvariable (label) im Training- und Testdatensatz bleibt, wie im Ursprungsdatensatz `df`.

2. Dataset.from_pandas():
train_df.reset_index(drop=True) und test_df.reset_index(drop=True): Zuerst wird der Index des DataFrames zurückgesetzt, damit keine alte Indexspalte übernommen wird. Das drop=True sorgt dafür, dass der alte Index vollständig entfernt wird.
Dataset.from_pandas(): Diese Methode konvertiert den Pandas DataFrame (train_df und test_df) in ein Dataset-Objekt aus der HuggingFace datasets-Bibliothek, das besser geeignet ist für den Einsatz in NLP-Pipelines.

In [15]:

# Aufteilen des DataFrames
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Konvertieren in Datasets
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# Erstellen eines DatasetDicts
dataset_dict = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


Modell und Trainer initialisieren:

In [38]:
from setfit import SetFitModel, SetFitTrainer
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Trainer konfigurieren:

In [42]:
from sentence_transformers.losses import CosineSimilarityLoss

import torch

# Sicherstellen, dass CUDA verfügbar ist und Gerät festlegen
training_device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


# Modell auf die GPU verschieben
model = model.to(training_device)

# Initialisieren des Trainers mit den Trainingsargumenten
trainer = SetFitTrainer(
    model=model, 
    train_dataset=dataset_dict['train'],
    eval_dataset=dataset_dict['test'],
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=10,  # Anzahl der Textpaarungen pro Beispiel
    num_epochs=1        # Anzahl der Epochen
)



TypeError: SetFitTrainer.__init__() got an unexpected keyword argument 'train_data'

In [37]:
trainer.train()


TypeError: 'numpy.bool_' object is not iterable

In [10]:
import datasets
metrics = trainer.evaluate()

metrics

***** Running evaluation *****


{'accuracy': 0.39473684210526316}

In [12]:
metrics

{'accuracy': 0.39473684210526316}